**Important note!** Before you turn in this lab notebook, make sure everything runs as expected:

- First, **restart the kernel** -- in the menubar, select Kernel$\rightarrow$Restart.
- Then **run all cells** -- in the menubar, select Cell$\rightarrow$Run All.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

# Part 2: Mining the web: Web APIs

We hope the preceding exercise was painful: even with tools to process HTML, it is rough downloading raw HTML and trying to extract information from it!

> Can you think of any other reasons why scraping websites for data in this way is not a good idea?

Luckily, many websites provide an application programming interface (API) for querying their data or otherwise accessing their services from your programs. For instance, Twitter provides a web API for gathering tweets, Flickr provides one for gathering image data, and Github for accessing information about repository histories.

These kinds of web APIs are much easier to use than, for instance, the preceding technique which scrapes raw web pages and then has to parse the resulting HTML. Moreover, there are more scalable in the sense that the web servers can transmit structured data in a less verbose form than raw HTML.

As a starting example, here is some code to look at the activity on Github related to the public version of our course's materials.

In [1]:
import requests

response = requests.get ('https://api.github.com/repos/cse6040/labs-fa17/events')

print ("==> .headers:", response.headers, "\n")

==> .headers: {'Date': 'Tue, 09 Jul 2019 08:46:37 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'GitHub.com', 'Status': '200 OK', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '59', 'X-RateLimit-Reset': '1562665597', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding', 'ETag': 'W/"8a2218ae40d0d7ed190ee8f19ee3afd3"', 'Last-Modified': 'Thu, 27 Jun 2019 14:10:46 GMT', 'X-Poll-Interval': '60', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'origin-

Note the `Content-Type` of the response:

In [2]:
print (response.headers['Content-Type'])

application/json; charset=utf-8


The response is in JSON format, which is an open format for exchanging semi-structured data. (JSON stands for **J**ava**S**cript **O**bject **N**otation.) JSON is designed to be human-readable and machine-readable, and maps especially well in Python to nested dictionaries. Let's take a look.

> See also [this tutorial](http://www.w3schools.com/json/) for a JSON primer. JSON is among _the_ universal formats for sharing data on the web; see, for instance, https://www.sitepoint.com/10-example-json-files/.

In [3]:
import json
print(type(response.json ()))
print(json.dumps(response.json()[:3], sort_keys=True, indent=2))

<class 'list'>
[
  {
    "actor": {
      "avatar_url": "https://avatars.githubusercontent.com/u/33994772?",
      "display_login": "kxu08",
      "gravatar_id": "",
      "id": 33994772,
      "login": "kxu08",
      "url": "https://api.github.com/users/kxu08"
    },
    "created_at": "2019-06-27T14:10:46Z",
    "id": "9906600823",
    "org": {
      "avatar_url": "https://avatars.githubusercontent.com/u/31073927?",
      "gravatar_id": "",
      "id": 31073927,
      "login": "cse6040",
      "url": "https://api.github.com/orgs/cse6040"
    },
    "payload": {
      "forkee": {
        "archive_url": "https://api.github.com/repos/kxu08/labs-fa17/{archive_format}{/ref}",
        "archived": false,
        "assignees_url": "https://api.github.com/repos/kxu08/labs-fa17/assignees{/user}",
        "blobs_url": "https://api.github.com/repos/kxu08/labs-fa17/git/blobs{/sha}",
        "branches_url": "https://api.github.com/repos/kxu08/labs-fa17/branches{/branch}",
        "clone_url": "https

**Exercise 0.** It should be self-evident that the JSON response above consists of a sequence of records, which we will refer to as _events_. Each event is associated with an _actor_. Write some code to extract a dictionary of all actors, where the key is the actor's login and the value is the actor's URL.

In [54]:
def extract_actors (json_github_events):
    """Given JSON records for events in a GitHub repo,
    returns a dictionary of the actors and their URLs.
    """
    #
    # YOUR CODE HERE
    #
    urls = {}
    for i in json_github_events :
        act = i['actor']['display_login']
        url = i['actor']['url']
        urls[act] = url
    return urls

In [60]:
actor_urls = extract_actors(response.json ())

for actor, url in actor_urls.items ():
    print ('{}: {}'.format(actor, url))
    assert url == "https://api.github.com/users/{}".format(actor)

kxu08: https://api.github.com/users/kxu08
steph-jung: https://api.github.com/users/steph-jung
sarojthapa60: https://api.github.com/users/sarojthapa60
tastyCanOfMalk: https://api.github.com/users/tastyCanOfMalk
nqtri: https://api.github.com/users/nqtri
ogckw: https://api.github.com/users/ogckw
pnkc: https://api.github.com/users/pnkc


**Exercise 1.** Write some code that goes to each actor's URL and determines their name. If an actor URL is invalid, that actor should not appear in the output.

In [69]:
def lookup_names (actor_urls):
    """Given a dictionary of (actor, url) pairs, looks up the JSON at
    the URL and extracts the user's name (if any). Returns a new
    dictionary of (actor, name) pairs.
    """
    #
    # YOUR CODE HERE
    #
    import re
    
    names = {}
    for actor, url in actor_urls.items():
        r = requests.get(url)
        if r is None : continue
        if re.search('application/json', r.headers['Content-Type']) is None: continue
        if 'name' not in r.json() : continue
        
        names[actor] = r.json()['name']
    return names

In [71]:
actor_names = lookup_names (actor_urls)

for actor, name in actor_names.items ():
    print ("{}: {}".format (actor, name))
    
assert actor_names['rvuduc'] == 'Rich Vuduc (personal account)'

kxu08: None
steph-jung: Stephanie Jung
sarojthapa60: Saroj Thapa
tastyCanOfMalk: Eddie
nqtri: Tri N
ogckw: Hans
pnkc: None


KeyError: 'rvuduc'

That's the end of this notebook. Processing JSON is fairly straightforward, because it maps very naturally to nested dictionaries in Python. You might search the web for other sources of JSON data, including [this one](https://www.yelp.com/dataset/challenge), and do your own processing!